# Proofread_nuclei

In [1]:
# libraries 1
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui import statebuilder,annotation,easyviewer,parser
from nglui.statebuilder import *
import json

import sys
import os
from datetime import datetime
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))
import authentication_utils as auth
import rootID_lookup as IDlook
sys.path.append(os.path.abspath("../synapses"))
import connectivity_utils

%matplotlib inline

In [5]:
df = pd.read_csv('../Output/soma_info_Aug2021ver2.csv', header=0)

In [8]:
df

,nucID,center_xyz,nuc_xyz,nuc_svID,nuc_rootID,soma_xyz,soma_svID,soma_rootID,vol,bbx_min,bbx_max
0,73115668492321345,"(64344, 155528, 2733)","(64384, 155376, 2719)",74244180477771882,0,"(64096, 155520, 2735)",74244180477821787,0,32625,"(64144, 155296, 2711)","(64544, 155760, 2756)"
1,72341681025843555,"(17024, 157440, 2665)","(16960, 157296, 2652)",72625768014154957,0,"(16768, 157376, 2693)",72625768014247001,0,33072,"(16832, 157232, 2639)","(17216, 157648, 2692)"
2,72694142819371754,"(40448, 196000, 1217)","(40352, 195888, 1211)",73401129131933600,0,"(40144, 195856, 1213)",73401129131935868,0,34440,"(40224, 195760, 1197)","(40672, 196240, 1238)"
3,72904562259788369,"(52608, 154120, 2647)","(52752, 154288, 2630)",73821967945732744,0,"(52320, 153968, 2663)",73821967945813511,0,35250,"(52368, 153920, 2624)","(52848, 154320, 2671)"
4,72481250081768943,"(27208, 86824, 2002)","(27376, 86896, 2005)",72975274937283119,0,"(26928, 86432, 1996)",72975274937264610,0,35875,"(27008, 86544, 1982)","(27408, 87104, 2023)"
...,...,...,...,...,...,...,...,...,...,...,...
17071,73114912242532398,"(61208, 110496, 1436)","(60624, 109584, 1417)",74101930422595776,0,"(59824, 110608, 1362)",74101999142110560,0,5548608,"(59856, 108960, 1351)","(62560, 112032, 1522)"
17072,72903943314735186,"(49280, 117624, 915)","(48496, 116896, 969)",73679992634339249,0,"(47616, 117376, 885)",73679992567212870,0,5691350,"(47664, 116336, 828)","(50896, 118912, 1003)"
17073,72270762727178308,"(14352, 124520, 3228)","(14112, 123456, 3312)",72483931349688631,0,"(13184, 124416, 3238)",72483931349546701,0,5712660,"(13216, 123328, 3093)","(15488, 125712, 3363)"
17074,72200462366935570,"(9072, 128072, 2373)","(8176, 127328, 2290)",72272962019227514,0,"(8032, 127680, 2353)",72272962086203920,0,5809152,"(8080, 126608, 2245)","(10064, 129536, 2501)"


In [ ]:
# color_column not supported for point annotation
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc_Aug = ImageLayerConfig(name = 'nuclei_Aug2021',
                                    source = auth.get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuclei_Aug2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Aug2021')['url'])      
nuc_Jul = ImageLayerConfig(name = 'nuclei_Jul2021',
                                    source = auth.get_cv_path('nuclei_map_Jul2021')['url'])      
nuc_seg_Jul = ImageLayerConfig(name = 'nuclei_Jul2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Jul2021')['url'])   
seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                    # selected_ids_column='nuc_segID',
                                    source = auth.get_cv_path('FANC_production_segmentation')['url'])      


df2 = df.assign(pt_position=[*zip(df.x, df.y, df.z)])
points = PointMapper(point_column='pt_position')
ann = AnnotationLayerConfig(name='nuc_center_Aug', mapping_rules=points, color='#32CB42',active = True,)      
sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, nuc_Jul, nuc_seg_Jul, ann],resolution=[4.3,4.3,45]) # view_kws=view_options

client,token = auth.get_client()

state = json.loads(sb.render_state(df2, return_as='json'))

out =auth.get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=auth.get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
out
# https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/692331708447824330
# https://github.com/htem/FANC_auto_recon/blob/main/proofreading/proofreading_utils.py



LINK=[]
k=200
minidfs = [df.loc[i:i+k-1, :] for i in range(0, len(df), k)]
for dftemp in minidfs:
    seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                        # selected_ids_column='nuc_segID',
                                        source = auth.get_cv_path('FANC_production_segmentation')['url'])                                 
    sb = StateBuilder(layers=[imgTokyo, seg, img, nuc, nuc_seg],resolution=[4.3,4.3,45]) # view_kws=view_options
    output = sb.render_state(dftemp) #return_as='html'
    LINK.append(output)

In [ ]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/nuc_links.csv', index=False, header=False)

In [ ]:
# color_column not supported for point annotation
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc_Aug = ImageLayerConfig(name = 'nuclei_Aug2021',
                                    source = auth.get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuclei_Aug2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Aug2021')['url'])      
nuc_Jul = ImageLayerConfig(name = 'nuclei_Jul2021',
                                    source = auth.get_cv_path('nuclei_map_Jul2021')['url'])      
nuc_seg_Jul = ImageLayerConfig(name = 'nuclei_Jul2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Jul2021')['url'])   
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                    active = True,
                                    source = auth.get_cv_path('FANC_production_segmentation')['url'])

df2 = df.assign(pt_position=[*zip(df.x, df.y, df.z)])
points = PointMapper(point_column='pt_position')
ann = AnnotationLayerConfig(name='nuc_center_Aug', mapping_rules=points, color='#32CB42')      
sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, nuc_Jul, nuc_seg_Jul, ann],resolution=[4.3,4.3,45]) # view_kws=view_options

client,token = auth.get_client()

state = json.loads(sb.render_state(df2, return_as='json'))

out =auth.get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=auth.get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
out
# https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/692331708447824330
# https://github.com/htem/FANC_auto_recon/blob/main/proofreading/proofreading_utils.py